In [1]:
import numpy as np
import csv
import os

In [2]:
def sample_data(source_file_path, target_file_path, sample_num):
    source_csv = open(source_file_path, 'r')
    dict_reader = csv.DictReader(source_csv)
    
    target_csv = open(target_file_path, 'w', newline='')
    file_header = ['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE', 
                  'MIN_LONG', 'MAX_LONG', 'MIN_LAT', 'MAX_LAT']
    dict_writer = csv.DictWriter(target_csv, file_header)
    dict_writer.writerow(dict(zip(file_header, file_header)))
    
    counter = 0
    
    for item in dict_reader:
        if counter >= sample_num:
            break
            
        
        if(len(eval(item['POLYLINE'])) == 0):
            continue
            
        polyline = np.array(eval(item['POLYLINE']))
        min_data = np.min(polyline, axis=0)
        max_data = np.max(polyline, axis=0)
        
        item['MIN_LONG'] = min_data[0]
        item['MAX_LONG'] = max_data[0]
        item['MIN_LAT'] = min_data[1]
        item['MAX_LAT'] = max_data[1]
        
        item['TRIP_ID'] = item['TRIP_ID'] + '\t'
        
        
        dict_writer.writerow(item)
        counter += 1
        
    source_csv.close()
    target_csv.close()
    

In [3]:
def convert_data_csv(source_file_path, target_file_path):
    source_csv = open(source_file_path, 'r')
    dict_reader = csv.DictReader(source_csv)
    
    target_csv = open(target_file_path, 'w', newline='')
    file_header = ['id;geom']
    writer = csv.writer(target_csv)
    
    writer.writerow(file_header)
    for item in dict_reader:
        row = []
        polyline = eval(item['POLYLINE'])
        trip_id = item['TRIP_ID'].strip()
        row.append('{};LINESTRING({} {}'.format(trip_id, polyline[0][0], polyline[0][1]))
        for i in range(1, len(polyline) - 1):
            row.append('{} {}'.format(polyline[i][0], polyline[i][1]))
        row.append('{} {})'.format(polyline[len(polyline) - 1][0], polyline[len(polyline) - 1][1]))
        writer.writerow(row)
        
    source_csv.close()
    target_csv.close()
        

In [4]:
def convert_data_txt(source_file_path, target_file_path):
    source_csv = open(source_file_path, 'r')
    dict_reader = csv.DictReader(source_csv)
    
    target_txt = open(target_file_path, 'w')
    file_header = 'id;geom;timestamp\n'
    
    target_txt.write(file_header)
    for (i, item) in enumerate(dict_reader):
        row = '{};LINESTRING('.format(i + 1)
        polyline = eval(item['POLYLINE'])
        
        for j in range(0, len(polyline)):
            row = row + '{} {},'.format(polyline[j][0], polyline[j][1])
        row = row[:len(row) - 1] + ');{}\n'.format(item['TIMESTAMP'])
        target_txt.write(row)
    target_txt.close()
    source_csv.close()

In [5]:
sample_data('data/train.csv', 'data/train_1000.csv', 1000)

In [6]:
convert_data_csv('data/train_1000.csv', 'data/trips.csv')

In [7]:
convert_data_txt('data/train_1000.csv', 'data/trips.txt')